# 多线程与多进程详解
　　参考：[Python 多线程与多进程](https://mp.weixin.qq.com/s/EFp5ZJ3ifih9XMA5ARoYeQ)<br>
  
  
  
## 一、引言

并发、并行、串行、同步、异步、阻塞、非阻塞、进程、线程、协程是并发编程中的常见概念，相似却也有却不尽相同，令人头痛，这一篇博文中我们来区分一下这些概念。

## 二、概念解释
### 2.1、进程与程序
　　程序: 指存储在磁盘上的包含有可执行机器代码和指令的文件。  
　　进程：是操作系统中正在运行的任务，是运行中的程序。
　　现代操作系统都是支持多个进程同时运行的系统。进程在系统中运行时，都拥有各自独立的虚拟地址空间，并在其中运行，因此某个进程的崩溃不会影响到其它进程的运行。系统内金承建的通信需要依赖系统内核（调度）来实现。  
　　根据进程特点，进程可分为3大类：交互进程、批处理进程、守护进程。  

#### 2.1.1、交互进程
　　交互进程是有shell启动的进程（Linux 系统），其既可以在前台运行，也可以在后台运行。交互进程在执行过程中，要求与用户进行交互操作，才能继续执行。  
#### 2.1.2、批处理进程
　　批处理进程是一个进程序列，负责按顺序启动其他进程。  
#### 2.1.3、守护进程
　　守护进程是执行特定功能或者执行系统相关任务的后台进程。其只是一个特殊进城，并不是内核的组成部分。守护进程既可以在系统启动时启动（直到系统关闭时关闭），也可以只在需要的时候才启动。  
#### 2.1.4、进程空间与虚拟内存
　　**交换分区**
　　系统中，交换分区实际上被用作内存的扩展，系统将暂时不会用到的数据写到交换分区中，在需要的时候再将磁盘（交换分区）中的内容读回内存。这样作，可以有效解决内存空间不足的问题，这种机制即被称为虚拟内存。  
　　虚拟内存技术用于解决多进程同时运行时内存空间不足的问题（各个进程都运行在各自的虚拟内存空间中）。此外还有如下功能：  
  1、巨大的寻址空间  
　　32位系统上，支持寻址4GB(232)的线性地址空间。用户看到的进程运行地址为虚拟地址，而不是物理地址。4GB的内存空间被分为用户空间和内核空间，用户程序通常情况下只能访问用户空间。用户程序只有通过系统调用才能进入内核空间。与用户空间的内容在进程切换时会发生变化不同，内核空间是由内核进行映射的，因此不会随着进程切换而发生变化。    
  2、共享虚拟内存  
　　某些情况下，不同进程之间需要进行数据共享，而各个进程又只能运行在自己的虚拟内存空间中，无法实现数据共享。虚拟内存提供了共享内存功能。通过共享内存，可以实现进程间的数据交换。    
  3、进程保护  
　　系统中，进程运行在自己的虚拟内存空间中，这样保证了进程运行的稳定性。   
#### 2.1.5、查看虚拟内存
　　vmstat 查看虚拟内存的使用情况的命令（vmstat不仅报告内存使用情况，还显示磁盘、CPU情况）。  
　　具体显示信息如下：  
　　procs:
+ r: 等待获得CPU时间以运行的进程数。  
+ b: 处于不可中断休眠的进程数。  
   
　　memory:  
+ swpd: 以使用虚拟内存大小。  
+ free: 空闲内存大小。  
+ buff: 用作磁盘缓冲的内存大小。  
+ cache: 用作缓存的内存大小。  
   
　　swap:  
+ si: 每秒从磁盘换入的内存大小。  
+ so: 每秒从内存写入的磁盘大小。  
   
　　io:
+ bi: 每秒从磁盘读取的块数，该数值可以用来表示读磁盘操作的情况。  
+ bo: 每秒写入磁盘的块数，该数值可以用来表示写磁盘操作的情况。  
   
　　system:
+ in: 每秒中断发生次数。  
+ cs: 每秒上下文切换次数。  
    
　　cpu:
+ us: 用户程序所占CPU的时间比例，该数值表示了用户进程的繁忙程度。  
+ sy: 运行内核代码所占的时间比例。  
+ id: CPU空闲时间所占的时间比例。  
+ wa: 用于等待I/O操作所占的时间比例。  
+ st: 抢占虚拟机运行所占的时间比例。  
　　注：vmstat 命令可获得比top命令更详细的系统运行信息。  
#### 2.1.6、进程内存
　　所有的进程都运行在自己的虚拟地址空间中，且每个进程都有自己的内存地址。   
　　在32位系统中，4GB的进程地址空间被分为用户空间和内核空间两部分。用户空间占据着 0-3GB(16进制表示即0xC0000000),而内核空间范围是3GB~4GB。  
　　对于一个进程而言，都会涉及3种不同的数据段，分别是代码段、数据段、堆栈段。  
　　**1、代码段**    
　　代码段是用于保存可执行文件的操作指令和程序定义的常量。为防止代码在运行时被其他进程修改，代码段将只允许读取，不可修改。多进程能够共享相同的代码段，即运行相同程序的进程共享代码段。  
　　**2、数据段**  
　　其位置紧接着代码段，分为初始化数据段和未初始化数据段（也被称为BSS段）。初始化数据段用于存放已经初始化的全局变量和程序的静态变量，而未初始化数据段用于保存未初始化的全局变量。  
　　**3、堆栈段**  
　　堆栈段中的**堆**用于存放进程中动态分配的内存地址。如C语言的malloc函数、C++的new函数分配的内存空间。当使用free函数（若内存是通过malloc函数分配）或delete函数释放内存时，分配的内存将从队中删除。  
　　**栈**用于保存程序中创建的临时变量。函数调用时，传递的参数也将被保存在栈中。栈具有先进先出的特点，适合于保存和恢复现场。可以将栈看作用于存放临时数据、进行数据交换的内存区域。栈的分配是向下分配的，因此最新分配的地址即为栈的结束地址（尾地址）。   
　　  
　　栈的大小受到操作系统的限制，因此从栈中获得的空间大小有限，而堆的大小只是受限于虚拟内存空间，因此使用堆可以获得较大的内存空间。堆得位置与数据段相邻。  
　　参考：结合《Linux编程技术详解》 page193的程序演示可深入理解。  
#### 2.1.7、进程标识
　　进程ID、用户ID、组ID  
　　**进程ID 与 父进程ID**  
　　Linux 系统使用进程ID(PID)来标识进程。除了 init 进程（pid为1）外，每个进程都有一个父进程。当父进程早于子进程结束时，子进程变为了“孤儿进程”，将被系统进程（init进程）收养。这时，init进程变为子进程的父进程。    
　　每个进程在创建的时候，系统都会分配一个进程ID给该进程。当进程ID达到系统允许最大值（<linux/threads.h>中有PID_MAX的定义，用以标识系统能够使用的最大PID数）时，系统将重新使用最小且当前未使用的PID号。  
　　可以使用ps命令获得当前系统正在运行的进程的PID信息，如:ps -aux。  
　　**用户标识（UID）与 有效用户标识（EUID）**  
　　当root用户创建新的用户时，会给每个用户分配一个用户ID(UID)。可以在/etc/passwd文件中查找到用户的UID信息。  
　　Linux系统每个进程都有两个ID: UID 与 EUID。当用户登录入系统时，系统会将UID与EUID设置为/etc/passwd文件中的UID。UID标识进程的创建者，只有进程的创建者和root用户才有权限对进程进行操作。    
　　EUID用于确定进程在任何给定的时刻，对资源和文件具有访问权限。一般情况下，UID与EUID是相同的。在某些特殊情况下，会出现这两标识不同的情况（如Linux中的passwd命令）。  
　　**组标识（GID）与 有效组标识（EGID）**  
　　进程的组标识（GID）、有效组标识（EGID）之间的关系与UID与EUID是类似的，区别在于UID和EUID用于描述用户对进程的访问权限的影响。GID和EGID用于描述用户组对进程访问权限的影响。  
#### 2.1.8、僵尸进程
　　僵尸进程（Zombie Process）是由于子进程退出后父进程没有使用 wait 函数收集子进程状态而产生的。僵尸进程是一个已经结束了的进程，不占用系统的内存资源，对CPU时间几乎没有任何影响，只是会占用进程表中的位置。由于进程表空间是有限的，Linux系统中产生的进程数是有限的，过多的僵尸进程会影响新进程的产生。  
　　僵尸进程的存在是为了给程序员和系统管理员提供一些重要的进程信息。这些信息包括进程如何结束，在运行过程中是否出现了错误等。如果没有僵尸进程，这些信息将随着进程的结束而消失。   
　　了解了僵尸进程产生的原因后，可以很容易地清除僵尸进程。Linux中，每个进程都有父进程。如果父进程被杀死，子进程则变为“孤儿进程”被init进程接管。如果子进程为僵尸进程的话，init进程将负责相应的清理工作。也就是说，只要杀死僵尸进程的父进程，就可以清除僵尸进程。   
  
  
  




### 2.1、并发与并行

在解释并发与并行之前，我们必须先明确：单个处理器（一个单核CPU）在某一个时刻只能处理一个线程 。

并发是指在同一个处理器上通过时间片轮转的方式在多个线程之间频繁切换，由于切换速度极快，所以看似多个线程似乎被同时执行，但实际上每一个时刻都只有一个线程被执行，其他的线程出于阻塞状态。

并行是指多个处理器在同一时刻同时处理了多个不同的线程，这才是真正意义的同时被执行。

如下图所示，线程A与线程B同在一个CPU内执行，且任一t时刻内，都只有一个线程（A或者B）被执行，所以线程A与线程B是并发执行的。线程C和线程D分别在两个CPU内执行，且在某一个t时刻内同时都在执行，所以线程C和线程D是并行的。

### 2.1、并行与串行

上面已经说到，并行是指多个任务同时执行，而 串行 是指多个任务时，各个任务按顺序执行，完成一个之后才能进行下一个。
所以，并发与并行是在某一时刻是否都在执行的区别。并行与串行是同时进行或一个结束才进行下一个的区别。

### 2.3、同步与异步

同步与异步的概念与消息的通知机制有关：

同步是指线程在访问某一资源时，获得了资源的返回结果之后才会执行其他操作，否则主动继续获取这一资源；

异步与同步相对，是指线程在访问某一资源时，无论是否取得返回结果，都进行下一步操作；当有了资源返回结果时，系统自会通知线程。

用一个比喻来说明：10多前的银行是没有业务取号的，我们去办理业务时，如果有很多人，那就先排队，然后关注着什么时候轮到自己，这就是同步；现在去银行，得先取一张小纸条，上面写着你的
业务号，轮到你的时候，银行会喊你，这就是异步。异步机制往往注册一个回调机制,在所等待的事件被触发时由触发机制(银行柜台业务员)通过某种机制(业务办理号码)找到等待该事件的人。
### 2.4、阻塞与非阻塞

阻塞是与非阻塞都是程序的一种运行状态。

线程在等待某个操作完成期间，自身无法继续执行别的操作，则称该线程在该操作上是 阻塞的。

线程在等待某个操作完成期间，自身可执行别的操作，则称该线程在该操作上是 非阻塞 的。

继续上面银行办理业务的例子，无论是10多年前的排队办理业务，还是现在的业务号办理业务，如果在我们在等待过程中，什么也不能做，那就是阻塞的；如果在等待过程中，可以做其他事情（看书
、玩手游），那就是非阻塞的。
同步和异步是个线程处理方式或手段，阻塞和非阻塞是线程的一种状态，两者并不相同也并不冲突。

同步、异步与阻塞非阻塞可以产生不同的组合：同步阻塞、同步非阻塞、异步阻塞、异步非阻塞。

还是银行办理业务的例子：如果排着队，且只能傻傻的排着队，看着什么时候到自己，那就是同步阻塞；如果排着队还能玩玩手机，偶尔抬头看看什么时候到自己，那就是同步非阻塞。如果是现在的
取票按业务号办理业务，拿到号码后就陷入懵逼状态，啥也不能做，直到银行根据业务号通知自己，那就是异步阻塞；如果拿到业务号之后，自己爱干嘛干嘛，那就是异步非阻塞。
### 2.5、进程 线程 协程

6.1 基本概念

进程是具有一定独立功能的程序关于某个数据集合上的一次运行活动,进程是系统进行资源分配和调度的一个独立单位，是资源（内存）分配的最小单位。每个进程都有自己的独立内存空间，不同进程通过进程间通信来通信。由于进程比较重量，占据独立的内存，所以上下文进程间的切换开销（栈、寄存器、虚拟内存、文件句柄等）比较大，但相对比较稳定安全。

线程是进程的一个实体,是CPU调度和分派的基本单位,它是比进程更小的能独立运行的基本单位.线程自己基本上不拥有系统资源,只拥有一点在运行中必不可少的资源(如程序计数器,一组寄存器和栈),但是它可与同属一个进程的其他的线程共享进程所拥有的全部资源。线程间通信主要通过共享内存，上下文切换很快，资源开销较少，但相比进程不够稳定容易丢失数据。

协程是一种用户态的轻量级线程，协程的调度完全由用户控制。协程拥有自己的寄存器上下文和栈。协程调度切换时，将寄存器上下文和栈保存到其他地方，在切回来的时候，恢复先前保存的寄存器上下文和栈，直接操作栈则基本没有内核切换的开销，可以不加锁的访问全局变量，所以上下文的切换非常快。

6.2 进程与线程

线程是指进程内的一个执行单元,也是进程内的可调度实体。线程与进程的区别:

1) 地址空间:线程是进程内的一个执行单元，进程内至少有一个线程，它们共享进程的地址空间，而进程有自己独立的地址空间；

2) 资源拥有:进程是资源分配和拥有的单位,同一个进程内的线程共享进程的资源；

3) 线程是处理器调度的基本单位,但进程不是；

4) 二者均可并发执行

5) 每个独立的线程有一个程序运行的入口、顺序执行序列和程序的出口，但是线程不能够独立执行，必须依存在应用程序中，由应用程序提供多个线程执行控制。

6.3 协程多与线程进行比较

1) 一个线程可以多个协程，一个进程也可以单独拥有多个协程，这样python中则能使用多核CPU。

2) 线程进程都是同步机制，而协程则是异步

3) 协程能保留上一次调用时的状态，每次过程重入时，就相当于进入上一次调用的状态。
  
## 三、线程与进程的区别
### 3.1、进程和线程区别
### 3.2、多进程和多线程的比较
## 四、Python全局解释器锁GIL
  
  
  